In [1]:
import tensorflow as tf
import tensorflow_io as tfio
import os 
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from methods_audio import data_handling
from methods_audio import data_augmentation
from methods_audio import denoising 
from methods_audio import model_performance_training
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, TimeMask, SpecCompose, SpecFrequencyMask
import random
from methods_audio import data_handling

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:104: UserWarning

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:104: UserWarning

In [2]:
data = data_handling.get_data2()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [3]:
len(data)

3

# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [4]:
data = data.map(data_handling.read_in_data) 

# 3. Get input for model training 

In [33]:
samples, labels = data_handling.extract_samples_labels(data)

In [32]:
def spectrogram_frequency_mask (spectrogram):
    print('make it here')
    augment = SpecCompose([
        SpecFrequencyMask(p=1),
    ])
    augmented_spectrogram = augment(spectrogram)
    print('not here')
    
    return augmented_spectrogram

In [7]:
def convert_to_spectrogram_temp(wave):
    # 1. Fast fourier transform #TODO: finish this 
    spectrogram = tf.signal.stft(wave, frame_length=256, frame_step=128)  # Paper: 'Automated detection of gunshots in tropical forests using CNN' 
    return spectrogram

def convert_to_spectrogram(wave):
    # 1. Fast fourier transform 
    spectrogram = tf.signal.stft(wave, frame_length=256, frame_step=128)  # Paper: 'Automated detection of gunshots in tropical forests using CNN' 
    # frame_length =  window length in samples --- frame_step = number of samples to step
    # 'Time frequency compromise': if window size is small: you get good time resolution in exchange of poor frequency resolution 

    # 2. Obtain the magnitude of the STFT
    spectrogram = tf.abs(spectrogram)

    # 3. Tranform it into appropiate format for deep learning model by adding the channel dimension (in this case 1)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [8]:
def transform_data (waves, type_transformation):
    transformed_signals = []

    if (type_transformation == 'spectrogram'):
        for wave in waves: 
            transformed_wave = convert_to_spectrogram(wave)
            transformed_signals.append(transformed_wave)
        
    elif(type_transformation == 'spectrogram_temp'): # TODO: finish 
        for wave in waves: 
            transformed_wave = convert_to_spectrogram_temp(wave)
            transformed_signals.append(transformed_wave)

    return transformed_signals

In [10]:
def spectrogram_augmentaion(samples, labels): 
 
    # 1. Transform data from time domain to frequency domain  
    spectrogram = transform_data(samples, type_transformation= 'spectrogram_temp')
    new_spects = []
    new_labels = []

    # 2. Do data augmentation in frequency domain 
    for spect, label in zip(spectrogram, labels): 
        print(type(spect))

        spect_augmented = spectrogram_frequency_mask(spect)
        print(type(spect_augmented))

        #spect_augmented = tfio.audio.freq_mask(spect, param=10)
        new_spects += [spect, spect_augmented]
        new_labels += [label]*2

        print('hey meli')
        print(type(spect_augmented))

    # 3. Transform back into time domain from frequency domain  
    #TODO     

    return new_spects, new_labels

Get data

In [34]:
from sklearn.model_selection import StratifiedKFold

splits = 2

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=123)   

for train, test in kfold.split(samples, labels):
    # 3. Split data and transform it from list of numpy.ndarray into numpy.ndarray of numpy.ndarray
    x_train = np.array(samples)[train.astype(int)]
    y_train = np.array(labels)[train.astype(int)]
    x_valid = np.array(samples)[test.astype(int)]
    y_valid = np.array(labels)[test.astype(int)]

    # 4. Transform numpy.ndarray into lists
    x_train_list = x_train.tolist() 
    y_train_list = y_train.tolist()
    x_valid_list = x_valid.tolist()

/var/folders/bz/rfsj8qqx5zsg0g1p0xdb1fkw0000gn/T/ipykernel_9802/484824578.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(samples)[train.astype(int)]
/var/folders/bz/rfsj8qqx5zsg0g1p0xdb1fkw0000gn/T/ipykernel_9802/484824578.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array(samples)[test.astype(int)]


In [35]:
x_train_list[:1]


[array([-0.08867513,  0.00858117,  0.21996477, ...,  0.04728521,
         0.14057186, -0.04004698], dtype=float32)]

In [36]:
x_train_list, y_train_list = spectrogram_augmentaion(x_train_list[:1], y_train_list[:1])

<class 'tensorflow.python.framework.ops.EagerTensor'>
make it here


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'copy'

In [ ]:
x_train_list[0]

In [ ]:
x_train_list[1]